<a href="https://colab.research.google.com/github/DeisyData/BIT_IA_Bootcamp/blob/main/S12_C1_Regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
# Load data
data = pd.read_excel('/content/marketing_extract_2X_Income.xlsx', sheet_name='Sheet1')

# Define features and target
X = data.drop(columns=['Income'])
y = data['Income']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1),
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5),
    'SVR': SVR(kernel='linear', C=1.0)
}

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['Model', 'MSE', 'MAE', 'R2'])

# Train and evaluate each model
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Append results
    results = pd.concat([results, pd.DataFrame({'Model': [name], 'MSE': [mse], 'MAE': [mae], 'R2': [r2]})], ignore_index=True)

# Sort results by R2
results = results.sort_values(by='R2', ascending=False)

# Display results
print("Model Performance:\n")
print(results)


<ipython-input-10-bfdc4e72f7e2>:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame({'Model': [name], 'MSE': [mse], 'MAE': [mae], 'R2': [r2]})], ignore_index=True)


Model Performance:

               Model           MSE           MAE        R2
6      Random Forest  1.901343e+08   8300.212494  0.614618
4         ElasticNet  2.015914e+08  10443.451905  0.591395
3              Lasso  2.015914e+08  10443.452469  0.591395
2              Ridge  2.015914e+08  10443.452430  0.591395
0  Linear Regression  2.015914e+08  10443.452436  0.591395
5                SVR  2.017091e+08  10433.786816  0.591157
1      Decision Tree  2.297573e+08   9712.207845  0.534306
7            XGBoost  3.232247e+08   8633.760672  0.344857
8     Neural Network  9.064015e+08  26460.824767 -0.837182


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
